# ImageRecorder
A [ImageRecorder](api.rst#ipywebrtc.webrtc.ImageRecorder) allows you to record a screenshot from any stream object, e.g. from:
 
 * [VideoStream](api.rst#ipywebrtc.webrtc.VideoStream)
 * [WidgetStream](api.rst#ipywebrtc.webrtc.WidgetStream)
 * [CameraStream](api.rst#ipywebrtc.webrtc.CameraStream)

In [ ]:
import ipywidgets as widgets
from ipywebrtc import ImageRecorder, VideoStream

In [ ]:
video = VideoStream.from_url('Big.Buck.Bunny.mp4')
video

Using the image recorder, you can take screenshot of the stream clicking the camera button

In [ ]:
image_recorder = ImageRecorder(stream=video)
image_recorder

In [ ]:
image_recorder.image

Or do it, programatically:

In [ ]:
image_recorder.recording = True

In [ ]:
image_recorder.autosave = False

In [ ]:
image_recorder.download()

In [ ]:
image_recorder.image.height

The data is PNG encoded (by default), so we show how to use PIL to read in the data

In [ ]:
import PIL.Image
import PIL.ImageFilter
import io
im = PIL.Image.open(io.BytesIO(image_recorder.image.value))

PIL Images display by default as image in the notebook. Calling the filter methods returns a new image which gets displayed directly.

In [ ]:
im.filter(PIL.ImageFilter.BLUR)

## Example with scikit image
We first convert the png encoded data to raw pixel values (as a numpy array).

In [ ]:
import numpy as np
im_array = np.array(im)
im_array

Now we can do easy manipulatios, such as reordering the channels (red, green, blue, alpha)

In [ ]:
PIL.Image.fromarray(im_array[...,::-1])

Or build a slightly more sophisticated example using scikit-image (run this notebook with a live kernel, such as mybinder for this to work)

In [ ]:
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage.color import rgb2gray
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage import filters


image = widgets.Image()

filter_options = [('roberts', roberts), ('sobel', sobel), ('scharr', scharr), ('prewitt', prewitt)]
filter_widget = widgets.ToggleButtons(options=filter_options)

def update_image(change):
    # turn into nparray
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)[...,:3] # no alpha
    
    # filter
    filter_function = filter_widget.value
    im_array_edges = adapt_rgb(each_channel)(filter_function)(im_array)
    im_array_edges = ((1-im_array_edges) * 255).astype(np.uint8)
    im_out = PIL.Image.fromarray(im_array_edges)
    
    # store in image widget
    f = io.BytesIO()
    im_out.save(f, format='png')
    image.value = f.getvalue()

image_recorder.observe(update_image, 'data')
filter_widget.observe(update_image, 'value')
widgets.VBox([filter_widget, video, widgets.HBox([image_recorder, image]), ])